# Feature Engineering

## ✅ DETAILED STEP-BY-STEP PLAN

__🔹 Task 2.1: Feature Engineering__
### Ticket 2.1.1: Create new features

- Use domain knowledge to engineer:
    - Water availability index: quantity / population served
    - Water safety score: categorical combination of water quality and quantity
    - Pump age: current year - construction_year (handle 0s/invalids)

### Ticket 2.1.2: Transform existing features

- Log-transform skewed features (e.g., population, amount_tsh)
    - Bin continuous variables:
        E.g., categorize amount_tsh into "low", "medium", "high"
    - Age into "new", "mid-age", "old"

### Ticket 2.1.3: Handle categorical features

- For low-cardinality: one-hot encoding
- For high-cardinality like funder, installer:
- Group rare values into “Other”
- Try Target Encoding / Frequency Encoding

### Ticket 2.1.4: Feature interaction terms

- Create interactions such as:
    region * pump age
    water_quality * quantity
- Use feature importance or SHAP to assess usefulness

### Ticket 2.1.5: Document all feature engineering

- Store all transformations in a script or notebook
- Maintain a markdown or .md file listing:
    Feature name
    Formula/logic
    Reason for inclusion


In [5]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from rapidfuzz import process, fuzz
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score



In [91]:
# Load dataset
data = pd.read_csv("../data/processed/Merged_Training_Set.csv")
df= data.copy()
print('Train dataset shape:', df.shape)

# Load GADM level-3 wards shapefile
gadm_shapefile_path = gpd.read_file("../data/external/gadm41_TZA_shp/gadm41_TZA_3.shp")

Train dataset shape: (59400, 41)


In [69]:
test_data = pd.read_csv("../data/raw/Test_Set_Values.csv")
df = test_data.copy()
df.shape

(14850, 40)

In [19]:
df.to_csv("../data/processed/unchanged_data_all.csv", index=False)

In [94]:
#import geopandas as gpd
#from fuzzywuzzy import fuzz, process

def impute_missing_coords(
    df_path: str,
    gadm_shapefile_path: str,
    score_threshold: int = 80
) -> pd.DataFrame:
    """
    Imputes missing or invalid latitude and longitude values using GADM level-3 shapefile centroids.
    
    Parameters:
    - df_path: Path to the CSV file containing the pump data.
    - gadm_shapefile_path: Path to the GADM level-3 shapefile (.shp file).
    - score_threshold: Minimum fuzzy match score to consider a ward match valid.

    Returns:
    - DataFrame with missing coordinates imputed.
    """

    # Step 1: Load data
    df = pd.read_csv(df_path)
    wards_gdf = gpd.read_file(gadm_shapefile_path)

    # Step 2: Normalize ward names
    wards_gdf.rename(columns={'NAME_3': 'ward', 'NAME_2': 'district'}, inplace=True)
    df['ward_clean'] = df['ward'].astype(str).str.lower()
    wards_gdf['ward_clean'] = wards_gdf['ward'].astype(str).str.lower()

    # Step 3: Replace invalid coordinates with NaN
    df['latitude'] = df['latitude'].replace([-2e-08, 0.0], pd.NA)
    df['longitude'] = df['longitude'].replace([0.0], pd.NA)

    # Step 4: Split valid and missing data
    df_valid = df[df['latitude'].notna() & df['longitude'].notna()].copy()
    df_missing = df[df['latitude'].isna() | df['longitude'].isna()].copy()
    print(f"🧊 Initial rows with missing coords: {len(df_missing)}")

    # Step 5: Fuzzy match ward names
    matches = []
    for ward in df_missing['ward_clean'].unique():
        match, score, _ = process.extractOne(
            query=ward,
            choices=wards_gdf['ward_clean'],
            scorer=fuzz.token_sort_ratio
        )
        matches.append({'ward_clean': ward, 'matched_ward': match, 'score': score})

    match_df = pd.DataFrame(matches)
    match_df = match_df[match_df['score'] >= score_threshold]

    # Step 6: Merge fuzzy matches with missing data
    df_missing = df_missing.merge(match_df, on='ward_clean', how='left')

    # Step 7: Merge with GADM shapes on matched ward name
    wards_unique = wards_gdf.drop_duplicates(subset=['ward_clean'])
    df_missing = df_missing.merge(
        wards_unique[['ward_clean', 'geometry']],
        left_on='matched_ward',
        right_on='ward_clean',
        how='left'
    )

    # Step 8: Impute missing lat/lon from geometry centroid
    def get_centroid_coords(geom):
        if geom and not geom.is_empty:
            return pd.Series([geom.centroid.y, geom.centroid.x])
        else:
            return pd.Series([pd.NA, pd.NA])

    df_missing[['latitude', 'longitude']] = df_missing['geometry'].apply(get_centroid_coords)

    # Step 9: Recombine with original valid data
    df_final_fixed = pd.concat([df_valid, df_missing], ignore_index=True)

    # Step 10: Final check
    print(f"✅ Final rows with missing lat/lon: {df_final_fixed['latitude'].isna().sum()}")
    print(f"✅ Final total rows: {len(df_final_fixed)} (original: {len(df)})")

    df_final_fixed = df_final_fixed.drop(['ward_clean','score','ward_clean_x','matched_ward', 'ward_clean_y', 'geometry'], axis=1) # 'ward_df_train','ward_df_gdf','score_x', 'score_y',

    return df_final_fixed

In [97]:
df_imputed = impute_missing_coords(
    df_path="../data/processed/Merged_Training_Set.csv",
    gadm_shapefile_path="../data/external/gadm41_TZA_shp/gadm41_TZA_3.shp"
)

🧊 Initial rows with missing coords: 1812
✅ Final rows with missing lat/lon: 0
✅ Final total rows: 59400 (original: 59400)


In [98]:
df = df_imputed.copy()
df.drop(columns=['recorded_by'], inplace=True)
df = df.fillna({col: 'missing' for col in df.select_dtypes(include=['object', 'category']).columns})

/var/folders/dy/n44rnxw55t54k5zb19w3gz7m0000gn/T/ipykernel_1031/2525687671.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna({col: 'missing' for col in df.select_dtypes(include=['object', 'category']).columns})


In [99]:
df.to_csv("../data/processed/imputed_lat_lon.csv", index=False)

In [32]:
df=pd.read_csv("../data/processed/imputed_lat_lon.csv")

In [100]:
from sklearn.neighbors import NearestNeighbors

def impute_subvillage_by_location(df):
    df = df.copy()
    
    # Separate known and unknown subvillages
    known = df[df['subvillage'].notnull() & df['latitude'].notnull() & df['longitude'].notnull()]
    unknown = df[df['subvillage'].isnull() & df['latitude'].notnull() & df['longitude'].notnull()]

    if not unknown.empty and not known.empty:
        # Fit NearestNeighbors on known locations
        nn = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
        nn.fit(known[['latitude', 'longitude']])
        
        # Find nearest known subvillage for each missing one
        distances, indices = nn.kneighbors(unknown[['latitude', 'longitude']])
        
        # Map nearest subvillages back to the original df
        nearest_subvillages = known.iloc[indices.flatten()]['subvillage'].values
        df.loc[unknown.index, 'subvillage'] = nearest_subvillages
    
    return df


In [101]:
df = impute_subvillage_by_location(df)

## Handling _funder_ and _installer_ columns, creating a new feature

In [102]:
from rapidfuzz import process, fuzz
import re

def clean_text(val):
    val = str(val).lower().strip()
    val = re.sub(r'[^a-z0-9\s]', '', val)
    return val

def fuzzy_clean_column(df, column, top_n=50, threshold=75, force_include=None):
    col_clean = f"{column}_clean"
    col_grouped = f"{column}_grouped"

    df[column] = df[column].fillna("missing")
    df[col_clean] = df[column].astype(str).apply(clean_text)

    top_values = df[col_clean].value_counts().head(top_n).index.tolist()
    if force_include:
        top_values += force_include
        top_values = list(set(top_values))

    def match_func(val):
        result = process.extractOne(val, top_values, scorer=fuzz.ratio)
        if result:
            match, score = result[0], result[1]
            if score >= threshold:
                return match
        return val

    df[col_grouped] = df[col_clean].apply(match_func)
    return df

def group_names(x):
    if pd.isnull(x):
        return x
    x = str(x).lower()
    if 'gov' in x:
        return 'government'
    if 'japan' in x or x in ['jica', 'jaica']:
        return 'japan'
    if 'german' in x:
        return 'germany'
    if 'village' in x:
        return 'village'
    return x

def combine_funder_installer_grouped(df):
    # Clean and fuzzy-match
    df = fuzzy_clean_column(df, "installer", top_n=50, threshold=75,
                            force_include=["hesawa", "government", "unicef", "jica", "rc church", "danida"])
    df = fuzzy_clean_column(df, "funder", top_n=50, threshold=75,
                            force_include=["hesawa", "government", "unicef", "jica", "rc church", "danida"])

    # Apply manual grouping logic
    df['installer_grouped'] = df['installer_grouped'].apply(group_names)
    df['funder_grouped'] = df['funder_grouped'].apply(group_names)

    # Manual corrections
    manual_installer_map = {
        'danid': 'danida',
        'commu': 'community',
        '0': 'unknown'
    }
    df['installer_grouped'] = df['installer_grouped'].replace(manual_installer_map)

    manual_funder_map = {
        'fini water': 'ministry of water',
        '0': 'unknown',
        'germany republi': 'germany',
        'adb': 'african development bank',
        # Add more rules as needed
    }
    df['funder_grouped'] = df['funder_grouped'].replace(manual_funder_map)

    # Create final combined column
    df['funder_installer_grouped'] = df['funder_grouped'] + "_" + df['installer_grouped']

    pair_counts = df['funder_installer_grouped'].value_counts()
    rare_threshold = 10
    rare_pairs = pair_counts[pair_counts < rare_threshold].index
    df['funder_installer_pair_grouped'] = df['funder_installer_grouped'].apply(
    lambda x: x if x not in rare_pairs else 'other'
)
    # Drop funder and installer original columns and intermediate columns, keep the cleaned and grouped ones
    df.drop(columns=['funder_clean', 'installer_clean','funder', 'installer','funder_installer_grouped','funder_grouped','installer_grouped'], inplace=True)
    return df


In [168]:
# frequency encoder function
def frequency_encode_column(df, column):
    freq_map = df[column].value_counts().to_dict()
    df[column + '_freq'] = df[column].map(freq_map)
    return df


In [103]:
df = combine_funder_installer_grouped(df)
#df = frequency_encode_column(df, 'funder_installer_pair_grouped')
#df = frequency_encode_column(df, 'funder_installer_grouped')

In [104]:
df.to_csv("../data/processed/imputed_latlon_funderInstaller.csv", index=False)

## Handling construction_year column

In [105]:
# To avoid negative pupm ages
df['subvillage_funder_installer'] = (
    df['subvillage'].astype(str) + "_" +
    df['funder_installer_pair_grouped'].astype(str)
)
#df = frequency_encode_column(df, 'subvillage_funder_installer')

In [106]:
def impute_construction_year(df):
    """
    Imputes construction_year using a hierarchical median strategy:
    1. funder_installer_grouped
    2. region
    3. global median
    """
    # Clean construction_year
    df['construction_year'] = df['construction_year'].replace(0, np.nan)
    df['unknown_construction_year'] = df['construction_year'].isna()

    # First impute using funder_installer_grouped
    med_funder_installer = (
        df.groupby('subvillage_funder_installer')['construction_year']
        .median()
        .rename('median_fi')
    )
    df = df.merge(med_funder_installer, on='subvillage_funder_installer', how='left')
    df['construction_year'] = df['construction_year'].fillna(df['median_fi'])
    df.drop(columns='median_fi', inplace=True)

    # Then impute remaining using region
    med_region = (
        df.groupby('region')['construction_year']
        .median()
        .rename('median_region')
    )
    df = df.merge(med_region, on='region', how='left')
    df['construction_year'] = df['construction_year'].fillna(df['median_region'])
    df.drop(columns='median_region', inplace=True)

    # Final fallback: global median
    global_median = df['construction_year'].median()
    df['construction_year'] = df['construction_year'].fillna(global_median)

    return df


In [107]:
df = impute_construction_year(df)

In [108]:
def calculate_pump_age(row): 
    construction_year = row['construction_year']
    try:
        recorded_year = pd.to_datetime(row['date_recorded']).year
    except:
        return np.nan
    
    if construction_year == 0 or pd.isna(construction_year) or pd.isna(recorded_year):
        return np.nan
    
    age = recorded_year - construction_year
    return age if age >= 0 else 0

df['pump_age'] = df.apply(calculate_pump_age, axis=1)

In [109]:
df.to_csv("../data/processed/imputed_latlon_fI_pumpage.csv", index=False)

In [60]:
df= pd.read_csv("../data/processed/imputed_latlon_funderInstaller_pumpage.csv")

In [46]:
# Dropping duplicated and lots of missing value columns
df.drop(columns=['num_private','region','public_meeting','scheme_name','payment','source','waterpoint_type_group','unknown_construction_year','subvillage_funder_installer'], inplace=True)

In [110]:
def compute_water_scores(df, population_col='population', quantity_col='quantity', quality_col='water_quality'):
    """
    Compute water_availability_index and water_safety_score with conservative mapping.

    Parameters:
    - df: DataFrame containing the data
    - population_col: name of the population column
    - quantity_col: name of the water quantity column (categorical)
    - quality_col: name of the water quality column (categorical)

    Returns:
    - df with new columns:
        - 'water_quantity_score'
        - 'water_quality_score'
        - 'water_availability_index'
        - 'water_safety_score'
    """

    # Conservative mapping for quantity (low = bad)
    quantity_map = {
        'dry': 1,
        'insufficient': 2,
        'seasonal': 3,
        'enough': 4,
        'unknown': 1  # conservative minimum
    }

    # Conservative mapping for quality (low = bad)
    quality_map = {
        'fluoride abandoned': 0,
        'salty abandoned': 0,
        'fluoride': 1,
        'salty': 1,
        'coloured': 2,
        'milky': 2,
        'soft': 3,
        'unknown': 0  # worst-case assumption
    }

    # Apply mappings
    df['water_quantity_score'] = df[quantity_col].map(quantity_map).fillna(1).astype(int)
    df['water_quality_score'] = df[quality_col].map(quality_map).fillna(0).astype(int)

    # Avoid division by zero: replace 0 population with np.nan to exclude from index
    population = df[population_col]#.replace(0, np.nan)

    # Compute availability index
    df['water_availability_index'] = df['water_quantity_score'] / population #(population+1)

    # Optional: fill NaNs with 0 if you want to handle them downstream
   # df['water_availability_index'] = df['water_availability_index'].fillna(0)

    # Composite water safety score:
    df['water_safety_sum'] = df['water_quality_score'] + df['water_quantity_score']
    #df['water_safety_prod'] = df['water_quality_score'] * df['water_quantity_score']

    return df


In [111]:
df = compute_water_scores(df)

In [112]:
import pandas as pd

def impute_population_by_group_median(df):
    """
    Impute population values (where 0 or NaN) using hierarchical group median logic.

    Parameters:
        df (pd.DataFrame): DataFrame containing population, region, waterpoint_type,
                           water_quantity_score, gps_height, latitude, and longitude.

    Returns:
        pd.DataFrame: Modified DataFrame with imputed 'population' and a 'population_imputed' flag.
    """
    # Step 1: Replace 0 with NaN
    df['population'] = df['population'].replace(0, pd.NA)
    
    # Step 2: Create helper bins
    df['gps_height_bin'] = pd.qcut(df['gps_height'], q=5, duplicates='drop')
    df['lat_lon_bin'] = pd.qcut(df['latitude'] * df['longitude'], q=10, duplicates='drop')

    # Step 3: Define groupings for imputation
    groupings = [
        ['region_code', 'water_quantity_score', 'waterpoint_type'],
        ['region_code', 'gps_height_bin'],
        ['region_code', 'lat_lon_bin'],
        ['region_code', 'waterpoint_type'],
        ['region_code']
    ]

    # Step 4: Apply group median imputation in order
    for group in groupings:
        median_vals = df.groupby(group)['population'].transform('median')
        df['population'] = df['population'].fillna(median_vals)

    # Step 5: Final fallback: global median
    df['population'] = df['population'].fillna(df['population'].median())

    return df


In [113]:
df = impute_population_by_group_median(df)

/Users/mariam/Documents/DC/HydroLogic/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/mariam/Documents/DC/HydroLogic/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/mariam/Documents/DC/HydroLogic/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/mariam/Documents/DC/HydroLogic/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/mariam/Documents/DC/HydroLogic/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out,

In [114]:
def impute_gps_by_ward(df):
    df=df.copy()

    df.loc[df['gps_height'] <=0, 'gps_height'] = np.nan 

    df['gps_height'] = df.groupby('ward')['gps_height'].transform(lambda x: x.fillna(x.median()))
    df['gps_height'].fillna(df['gps_height'].median(), inplace=True)

    return df

In [115]:
df = impute_gps_by_ward(df)

/Users/mariam/Documents/DC/HydroLogic/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/mariam/Documents/DC/HydroLogic/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/mariam/Documents/DC/HydroLogic/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/mariam/Documents/DC/HydroLogic/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/mariam/Documents/DC/HydroLogic/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out,

In [116]:
print(df.shape)
print(df.columns)

(59400, 48)
Index(['id', 'amount_tsh', 'date_recorded', 'gps_height', 'longitude',
       'latitude', 'wpt_name', 'num_private', 'basin', 'subvillage', 'region',
       'region_code', 'district_code', 'lga', 'ward', 'population',
       'public_meeting', 'scheme_management', 'scheme_name', 'permit',
       'construction_year', 'extraction_type', 'extraction_type_group',
       'extraction_type_class', 'management', 'management_group', 'payment',
       'payment_type', 'water_quality', 'quality_group', 'quantity',
       'quantity_group', 'source', 'source_type', 'source_class',
       'waterpoint_type', 'waterpoint_type_group', 'status_group',
       'funder_installer_pair_grouped', 'subvillage_funder_installer',
       'unknown_construction_year', 'pump_age', 'water_quantity_score',
       'water_quality_score', 'water_availability_index', 'water_safety_sum',
       'gps_height_bin', 'lat_lon_bin'],
      dtype='object')


In [117]:
df.to_csv("../data/processed/imputed_latlon_fI_pumpage_pop_gps.csv", index=False)

In [90]:
df.to_csv("../data/processed/test_imputed_latlon_fI_pumpage_pop_gps.csv", index=False)

In [146]:
df = pd.read_csv("../data/processed/pop_imputed_feature_eng_all.csv")

In [185]:
# Drop columns that have high number of missing values
df.drop(columns=['date_recorded','scheme_management','permit','scheme_name','wpt_name','public_meeting','unknown_construction_year'], inplace=True)
df.shape

(59400, 40)

In [186]:
# Drop columns that have duplicate information
df.drop(columns=['region','quantity','source','extraction_type','quality_group','water_quality','management','funder_installer_pair_grouped','subvillage_funder_installer'], inplace=True)
df.shape

(59400, 31)

In [ ]:
# For test data Drop columns that have duplicate information
df.drop(columns=['payment','waterpoint_type_group','quantity','source','extraction_type','quality_group','water_quality','management','funder_installer_pair_grouped','subvillage_funder_installer'], inplace=True)
df.shape

In [187]:
df.to_csv("../data/processed/pop_cat_imputed_cleaned_feature_eng.csv", index=False)

In [151]:
df.to_csv("../data/processed/pop_test_feature_eng_cleaned.csv", index=False)

In [188]:
from sklearn.cluster import KMeans
import numpy as np

coords = df[['latitude', 'longitude']]
kmeans = KMeans(n_clusters=10, random_state=42)
df['geo_cluster'] = kmeans.fit_predict(coords)


In [189]:
df['lat_bin'] = (df['latitude'] * 10).round(0)
df['lon_bin'] = (df['longitude'] * 10).round(0)
df['location_bucket'] = df['lat_bin'].astype(str) + "_" + df['lon_bin'].astype(str)

In [190]:
df['lat_long_interaction'] = df['latitude'] + df['longitude']

In [191]:
# This interaction captures how well an old pump holds up under resource strain.
df['pumpage_safety_inter'] = df['pump_age'] * df['water_safety_sum']

quantity_score: Measures how much water is available — crucial to functionality.
extraction_type: Defines the mechanism used to get water (handpump, electric, etc.).
These combine to explain mechanical suitability or mismatch, e.g.:
Low quantity + high-demand extraction = likely failure
High quantity + appropriate extraction = success

In [192]:
df['extraction_type_class'], _ = pd.factorize(df['extraction_type_class'])
df['quantity_extraction_inter'] = df['water_quantity_score'] * df['extraction_type_class']

In [193]:
print(df.shape)
print(df.columns)

(59400, 38)

In [194]:
df.to_csv("../data/processed/pop_cat_train_new_feature.csv", index=False)

In [158]:
df.to_csv("../data/processed/pop_test_new_feature.csv", index=False)

In [205]:
df = pd.read_csv("../data/processed/pop_cat_train_new_feature.csv")

In [206]:
# Drop duplicate columns
df.drop(columns=['latitude','longitude','funder_grouped','installer_grouped','extraction_type_group','source_type','location_bucket'], inplace=True)
df.shape

(59400, 31)

In [207]:
df.drop(columns=['extraction_type_class','construction_year','pump_age','water_availability_index','water_safety_sum','lat_bin','lon_bin'], inplace=True)
df.shape

(59400, 24)

In [208]:
print(df.shape)
print(df.columns)

(59400, 24)
Index(['amount_tsh', 'gps_height', 'basin', 'subvillage', 'region_code',
       'district_code', 'lga', 'ward', 'population', 'management_group',
       'payment_type', 'source_class', 'waterpoint_type', 'status_group',
       'funder_installer_pair_grouped_freq',
       'subvillage_funder_installer_freq', 'water_quantity_score',
       'water_quality_score', 'gps_height_bin', 'lat_lon_bin', 'geo_cluster',
       'lat_long_interaction', 'pumpage_safety_inter',
       'quantity_extraction_inter'],
      dtype='object')


__# After feature selection the following can also be dropped:__

In [ ]:
df.drop(columns=['source_class','geo_cluster','district_code'], inplace=True)
df.shape

In [197]:
df.to_csv("../data/processed/pop_cat_train_new_feature_cleaned.csv", index=False)

In [95]:
df.to_csv("../data/processed/test_cleandropped_feature_.csv", index=False)

### Encoding categorical features

In [198]:
categorical_columns = [
    'subvillage','lga','ward','payment_type','basin','management_group','waterpoint_type'
]

for column in categorical_columns:
    df[column], _ = pd.factorize(df[column])

In [199]:
# Map the categorical values to numerical values
status_mapping = {'non functional': 0, 'functional needs repair': 1, 'functional': 2}
df['status_group'] = df['status_group'].map(status_mapping)

In [ ]:
print(df.shape)
print(df.columns)

(14850, 24)
Index(['id', 'amount_tsh', 'gps_height', 'basin', 'subvillage', 'region_code',
       'district_code', 'lga', 'ward', 'population', 'management_group',
       'payment_type', 'source_class', 'waterpoint_type',
       'funder_installer_pair_grouped_freq',
       'subvillage_funder_installer_freq', 'gps_height_bin', 'lat_lon_bin',
       'water_quantity_score', 'water_quality_score', 'geo_cluster',
       'lat_long_interaction', 'pumpage_safety_inter',
       'quantity_extraction_inter'],
      dtype='object')


In [163]:
df.to_csv("../data/processed/pop_test_encoded_.csv", index=False)

In [200]:
df.to_csv("../data/processed/pop_train_encoded_.csv", index=False)

Still possible to improve features:
- impute construction_year using gps_height
- impute population based on ward data
- Robust scaling
- Extraction_type and waterpoint_type

In [ ]:
ward_pop = pd.read_csv("../data/external/tza_pop_popn_nbs_baselinedata.csv") 
# Standardize column names
ward_pop.rename(columns={
    'Ward_Name': 'ward',
    'total_both': 'ward_population_2012'
}, inplace=True)

ward_pop['ward'] = ward_pop['ward'].astype(str).str.strip().str.lower()
#========
pop_df = data.copy()
pop_df['ward'] = pop_df['ward'].astype(str).str.strip().str.lower()
pop_df['subvillage'] = pop_df['subvillage'].astype(str).str.strip().str.lower()
#========
def impute_population_by_ward(pop_df, ward_pop, ward_col='ward', subvillage_col='subvillage',
                               pop_col='population', ward_name_col='ward', ward_pop_col='ward_population_2012'):
    """
    Imputes missing or zero population values in the main dataset `df` using ward-level population data.

    Parameters:
    - pop_df: main DataFrame (must contain ward, subvillage, population columns)
    - ward_pop: DataFrame with ward-level population (2012 census or similar)
    - ward_col: column name for ward in `df`
    - subvillage_col: column name for subvillage in `df`
    - pop_col: column name for population in `df`
    - ward_name_col: column in `ward_pop_df` with ward names
    - ward_pop_col: column in `ward_pop_df` with total population

    Returns:
    - Updated DataFrame with imputed population values
    """

    # Step 1: Standardize ward names
    pop_df[ward_col] = pop_df[ward_col].astype(str).str.strip().str.lower()
    ward_pop[ward_name_col] = ward_pop[ward_name_col].astype(str).str.strip().str.lower()

    # Step 2: Merge ward population into main df
    pop_df = pop_df.merge(
        ward_pop[[ward_name_col, ward_pop_col]],
        how='left',
        left_on=ward_col,
        right_on=ward_name_col
    ).rename(columns={ward_pop_col: 'ward_population_2012'})

    # Step 3: Count number of subvillages in each ward
    subvillage_counts = pop_df.groupby(ward_col)[subvillage_col].nunique().rename('subvillage_count')
    pop_df = pop_df.merge(subvillage_counts, on=ward_col, how='left')

    # Step 4: Impute population where missing or 0
    def impute(row):
        pop = row[pop_col]
        ward_pop = row['ward_population_2012']
        sv_count = row['subvillage_count']

        if pd.isna(pop) or pop == 0:
            if pd.notna(ward_pop) and sv_count > 0:
                return round(ward_pop / sv_count)
        return pop

    pop_df[pop_col] = pop_df.apply(impute, axis=1).astype('Int64')  # Use Int64 to allow nulls

    # Optional: drop helper columns if not needed
    # df = df.drop(columns=['ward_population_2012', 'subvillage_count', ward_name_col])

    return pop_df
#==========
pop_df = impute_population_by_ward(pop_df, ward_pop)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd

columns_to_encode = ['basin','management_group','source_class','waterpoint_type']

# Define encoder
encoder = ColumnTransformer(
    transformers=[('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), columns_to_encode)],
    remainder='passthrough'  # keeps other columns
)

# Fit and transform
df_encoded = pd.DataFrame(encoder.fit_transform(df), columns=encoder.get_feature_names_out())

# Optionally, set index to match original df
df_encoded.index = df.index


In [ ]:
def calculate_pump_age(row): #, min_year=2010, max_year=2015
    construction_year = row['construction_year']
    try:
        recorded_year = pd.to_datetime(row['date_recorded']).year
    except:
        return np.nan
    
    #if recorded_year < min_year or recorded_year > max_year:
     #   recorded_year = np.nan
    
    if construction_year == 0 or pd.isna(construction_year) or pd.isna(recorded_year):
        return np.nan
    
    age = recorded_year - construction_year
    return age if age >= 0 else 0

df['pump_age'] = df.apply(calculate_pump_age, axis=1)

In [ ]:
# Calculate median pump_age for each group
group_medians = df.groupby('subvillage_funder_installer')['pump_age'].median()

# Define a function to impute
def impute_pump_age(row):
    if pd.isna(row['pump_age']):
        group = row['subvillage_funder_installer']
        if group in group_medians and not pd.isna(group_medians[group]):
            return group_medians[group]
        else:
            # fallback median for entire dataset if group median is not available
            return df['pump_age'].median()
    else:
        return row['pump_age']

df['pump_age'] = df.apply(impute_pump_age, axis=1)

In [ ]:
def engineer_water_availability_index(df, 
                                      quantity_col='quantity', 
                                      population_col='population',
                                      score_col='quantity_score',
                                      wai_col='water_availability_index',
                                      log_col='wai_log',
                                      bin_col='wai_bin',
                                      do_log=True,
                                      do_bin=True,
                                      n_bins=4):
    """
    Engineer Water Availability Index from quantity and population.

    Parameters:
    df             : DataFrame with quantity and population columns
    quantity_col   : Column name for water quantity (categorical)
    population_col : Column name for population served
    score_col      : New column for numeric score of quantity
    wai_col        : Column name for final water availability index
    log_col        : Optional log-transformed column
    bin_col        : Optional binned version of WAI
    do_log         : Whether to create log-transformed WAI
    do_bin         : Whether to create binned WAI
    n_bins         : Number of quantile bins for binned WAI

    Returns:
    Modified DataFrame with new feature columns
    """

    quantity_map = {
        'unknown': 0,
        'dry': 1,
        'insufficient': 2,
        'seasonal': 3,
        'enough': 4
    }

    # Map quantity to ordinal score
    df[score_col] = df[quantity_col].map(quantity_map)

    # Avoid divide-by-zero and invalid values
    df['population_fixed'] = df[population_col].replace(0, np.nan)

    # Calculate Water Availability Index
    df[wai_col] = df[score_col] / df['population_fixed']

    # Optional log transform
  #  if do_log:
   #     df[log_col] = np.log1p(df[wai_col])

    # Optional binning (e.g. for tree-based models)
    #if do_bin:
     #   df[bin_col] = pd.qcut(df[wai_col], q=n_bins, duplicates='drop',  labels=[f'wai_bin_{i+1}' for i in range(n_bins)])

    return df

df = engineer_water_availability_index(df)